In [2]:
import utils
inp = utils.get_input(2019, 10)[:-1]
# inp = """.#..#
# .....
# #####
# ....#
# ...##"""
# inp = """#.#...#.#.
# .###....#.
# .#....#...
# ##.#.#.#.#
# ....#.#.#.
# .##..###.#
# ..#...##..
# ..##....##
# ......#...
# .####.###."""
# inp = """.#..##.###...#######
# ##.############..##.
# .#.######.########.#
# .###.#######.####.#.
# #####.##.#.##.###.##
# ..#####..#.#########
# ####################
# #.####....###.#.#.##
# ##.#################
# #####.##.###..####..
# ..######..##.#######
# ####.##.####...##..#
# .#####..#.######.###
# ##...#.##########...
# #.##########.#######
# .####.#.###.###.#.##
# ....##.##.###..#####
# .#.#.###########.###
# #.#.#.#####.####.###
# ###.##.####.##.#..##"""
print(inp)

Found cached input for 2019 day 10
#..#.#.#.######..#.#...##
##.#..#.#..##.#..######.#
.#.##.#..##..#.#.####.#..
.#..##.#.#..#.#...#...#.#
#...###.##.##..##...#..#.
##..#.#.#.###...#.##..#.#
###.###.#.##.##....#####.
.#####.#.#...#..#####..#.
.#.##...#.#...#####.##...
######.#..##.#..#.#.#....
###.##.#######....##.#..#
.####.##..#.##.#.#.##...#
##...##.######..##..#.###
...###...#..#...#.###..#.
.#####...##..#..#####.###
.#####..#.#######.###.##.
#...###.####.##.##.#.##.#
.#.#.#.#.#.##.#..#.#..###
##.#.####.###....###..##.
#..##.#....#..#..#.#..#.#
##..#..#...#..##..####..#
....#.....##..#.##.#...##
.##..#.#..##..##.#..##..#
.##..#####....#####.#.#.#
#..#..#..##...#..#.#.#.##


In [6]:
import math
import numpy as np
from copy import deepcopy
from operator import itemgetter
from pprint import pprint
from itertools import repeat, chain, cycle
from functools import partial
from collections import defaultdict, OrderedDict


sign = partial(math.copysign, 1)


class memoize(dict):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        return self[args]

    def __missing__(self, key):
        result = self[key] = self.func(*key)
        return result


rows = inp.split('\n')
m = list(map(list, rows))
asteroids = []

for x in range(len(m)):
    for y in range(len(m[0])):
        if m[y][x] == '#':
            asteroids.append((x, y))

# print(asteroids)

# @memoize
def perimeter(a, d):
    l = []
    for x in range(max(a[0] - d, 0), min(a[0] + d + 1, len(m))):
        for y in range(max(a[1] - d, 0), min(a[1] + d + 1, len(m))):
            l.append((x, y))
    return [x for x in l if x in asteroids]

# @memoize
def line(a, b):
    def find_min(n, step):
        if step > 0:
            while (n - step) >= 0:
                n -= step
        return n

    k_x = b[0] - a[0]
    k_y = b[1] - a[1]
    m_x = min(a[0], b[0])
    m_y = min(a[1], b[1])
    scale = max(abs(k_x), abs(k_y))
    l = []
    
    x_sign = int(sign(b[0] - a[0]))
    y_sign = int(sign(b[1] - a[1]))

    x_add = repeat(scale * a[0]) if abs(k_x) < abs(k_y) else []
    y_add = repeat(scale * a[1]) if abs(k_y) < abs(k_x) else []
    for x_, y_ in zip(
        chain(range(scale * a[0], scale * b[0] + x_sign, k_x or x_sign), x_add),
        chain(range(scale * a[1], scale * b[1] + y_sign, k_y or y_sign), y_add)
    ):
        l.append((x_ / scale, y_ / scale))
    return [x for x in l if x != a]


def clear(a, b):
    on_line = line(a, b)
    for i in range(len(m)):
        on_perimeter = perimeter(a, i)
        if set(on_perimeter).intersection(on_line):
            if b in on_perimeter:
                return True
            return False
    return False


def angles(a):
    res = OrderedDict({})
    for y in range(0, len(m)):
        for x in range(0, len(m)):
            if a != (x, y):
                if a[1] == y:
                    res[(x,y)] = 90.0 if x > a[0] else 270.0
                elif a[0] == x:
                    res[(x,y)] = 0.0 if y < a[1] else 180.0
                else:
                    t = math.degrees(math.atan2((x - a[0]), (a[1] - y)))
                    if t < 0:
                        t += 360
                    res[(x,y)] = t
    return res


clockwise = angles((11,19))
clockwise = {k: v for k, v in clockwise.items() if k in asteroids}
clockwise = OrderedDict(sorted(clockwise.items(), key=itemgetter(1)))


def manhattan(a, b):
    return abs(b[0] - a[0]) + abs(b[1] - a[1])
    

def part_2(start):
    l = []
    queue = deepcopy(clockwise)
    for a in cycle(clockwise):
        if (a in queue) and queue != start:
            on_line = [x for x in line(start, a) if x in queue]
            dists = map(lambda t: manhattan(start, t), on_line)
            if manhattan(start, a) <= min(dists):
                if clear(start, a):
#                     print(len(queue))
                    l.append(a)
                    del queue[a]
        if len(l) >= 200:
            break
    return l

res = part_2((11,19))[199]
print(res[0] * 100 + res[1])

815


In [113]:
detect = defaultdict(list)
for a in asteroids:
    for b in asteroids:
        if (a != b) and clear(a, b):
            detect[a].append(b)

pprint([(k, len(v)) for k, v in detect.items() if len(v) == max(map(len, detect.values()))])


[((11, 19), 253)]
